In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Dropout, GlobalAveragePooling1D, Dense
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from typing import Tuple
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import gc

In [2]:
df_embed = pd.read_csv('./all_AF/new/embeddings_clustered_short.csv', header=0).drop(columns=['seq'])

In [3]:
def make_train_test_dataset(df: pd.DataFrame, test_size: float = 0.2) -> Tuple[np.ndarray, np.ndarray,
                                                                               np.ndarray, np.ndarray]:
    df_train, df_test = train_test_split(df, test_size=test_size, random_state=24)

    train_labels = df_train.pop('label').to_numpy()
    test_labels = df_test.pop('label').to_numpy()

    X_train = np.swapaxes(np.expand_dims(df_train, axis=1), 1, 2)
    y_train = np.expand_dims(train_labels, axis=1)

    X_test = np.swapaxes(np.expand_dims(df_test, axis=1), 1, 2)
    y_test = np.expand_dims(test_labels, axis=1)

    return X_train, y_train, X_test, y_test

In [4]:
X_train, y_train, X_test, y_test = make_train_test_dataset(df_embed)
del(df_embed)
gc.collect()

109

In [5]:
X_train.shape

(192204, 1024, 1)

In [6]:
model = Sequential([
        Conv1D(32, kernel_size=8, data_format='channels_last', activation='relu', input_shape=(1024,1)),
        BatchNormalization(),
        MaxPooling1D(),
        Conv1D(16, kernel_size=8, data_format='channels_last', activation='relu'),
        BatchNormalization(),
        MaxPooling1D(),
        Conv1D(4, kernel_size=8, data_format='channels_last', activation='relu'),
        BatchNormalization(),
        MaxPooling1D(),
        Dropout(0.3),
        GlobalAveragePooling1D(),
        Dense(1, activation='sigmoid')])

In [7]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1017, 32)          288       
                                                                 
 batch_normalization (BatchN  (None, 1017, 32)         128       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 508, 32)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 501, 16)           4112      
                                                                 
 batch_normalization_1 (Batc  (None, 501, 16)          64        
 hNormalization)                                                 
                                                        

In [8]:
model.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=10
)


Epoch 1/10
3004/3004 [==============================] - 324s 107ms/step - loss: 0.4113 - accuracy: 0.8302
Epoch 2/10
3004/3004 [==============================] - 318s 106ms/step - loss: 0.3719 - accuracy: 0.8532
Epoch 3/10
3004/3004 [==============================] - 308s 103ms/step - loss: 0.3649 - accuracy: 0.8587
Epoch 4/10
3004/3004 [==============================] - 314s 105ms/step - loss: 0.3595 - accuracy: 0.8612
Epoch 5/10
3004/3004 [==============================] - 322s 107ms/step - loss: 0.3552 - accuracy: 0.8636
Epoch 6/10
3004/3004 [==============================] - 426s 142ms/step - loss: 0.3522 - accuracy: 0.8646
Epoch 7/10
3004/3004 [==============================] - 437s 145ms/step - loss: 0.3501 - accuracy: 0.8663
Epoch 8/10
3004/3004 [==============================] - 440s 146ms/step - loss: 0.3488 - accuracy: 0.8675
Epoch 9/10
3004/3004 [==============================] - 470s 156ms/step - loss: 0.3458 - accuracy: 0.8688
Epoch 10/10
3004/3004 [=======================

In [9]:
score = model.evaluate(X_test, y_test, verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.5542090535163879
Test accuracy: 0.7502445578575134


In [77]:
model.save("tf_embed_cnn_10epochs.h5", save_format='h5')